In [ ]:
import pandas as pd

In [ ]:
# Se cargan los datos. 
CollegeScoreCard=pd.read_csv('CollegeScorecard.csv', sep=',', encoding = 'utf-8')
# Cantidad de datos y número de variables
CollegeScoreCard.shape

In [ ]:
# Depuracion y eleccion de variables
data=CollegeScoreCard[CollegeScoreCard["CURROPER"]!=0]
data=data[["INSTNM","CONTROL","ADM_RATE","ADM_RATE_ALL","COSTT4_A","COSTT4_P","PCTPELL","PCTFLOAN"]]
data[data["COSTT4_A"].isnull() & data["COSTT4_P"].isnull()]
data=data[data["COSTT4_A"].notnull() | data["COSTT4_P"].notnull()]
data

In [ ]:
data[data['INSTNM'].str.contains('niversity') & data["COSTT4_P"].notnull()]
data[~data['INSTNM'].str.contains('niversity') & ~data['INSTNM'].str.contains('ollege') & ~data['INSTNM'].str.contains('nstitu')]
data[data['INSTNM'].str.contains('niversity') | data['INSTNM'].str.contains('ollege') | data['INSTNM'].str.contains('nstitu') | data['INSTNM'].str.contains('chool') | data['INSTNM'].str.contains('enter')]
data[~data['INSTNM'].str.contains('niversity') & ~data['INSTNM'].str.contains('ollege') & ~data['INSTNM'].str.contains('nstitu') & ~data['INSTNM'].str.contains('chool') & ~data['INSTNM'].str.contains('enter')]
data[~data['INSTNM'].str.contains('niversity') & ~data['INSTNM'].str.contains('ollege') & ~data['INSTNM'].str.contains('nstitu') & ~data['INSTNM'].str.contains('chool') & ~data['INSTNM'].str.contains('enter') & ~data['INSTNM'].str.contains('cadem')]
data[~data['INSTNM'].str.contains('niversity') & ~data['INSTNM'].str.contains('ollege') & ~data['INSTNM'].str.contains('nstitu') & ~data['INSTNM'].str.contains('chool') & ~data['INSTNM'].str.contains('enter') & ~data['INSTNM'].str.contains('cadem')  & ~data['INSTNM'].str.contains('CET')]
data[data['INSTNM'].str.contains('niversity') | data['INSTNM'].str.contains('ollege') | data['INSTNM'].str.contains('nstitu') ]

In [ ]:
data=data[data['INSTNM'].str.contains('niversity') | data['INSTNM'].str.contains('ollege') | data['INSTNM'].str.contains('nstitu') ]

In [ ]:
data[data["COSTT4_A"].notnull()]
data=data[data["COSTT4_A"].notnull()]
data

In [ ]:
data=data.drop(['COSTT4_P'],axis=1)
data

In [ ]:
def trans(p):
    p_mod=''
    if p == 1:
        return "PUBLIC"
    elif p == 2:
        return "PRIVATE NONPROFIT"
    else:
        return "PRIVATE FOR-PROFIT"

In [ ]:
data["CONTROL"]=data["CONTROL"].apply(trans)
data

In [ ]:
data['CONTROL'].value_counts()
data.describe()

In [ ]:
data[data['ADM_RATE_ALL'].isnull() ]
data[data['PCTPELL'].isnull() ]
data[data['PCTFLOAN'].isnull() ]

In [ ]:
data=data.drop([2124,2582])
data